# PTUSE observation report

In [ ]:
print 'Report Started:'
!date

## Files to be processed

In [ ]:
directory = 

## Set up environment

In [ ]:
%pylab inline

In [ ]:
import os
import sys
import numpy as np
from matplotlib import cm
from matplotlib import colors
import psrchive as psr
from coast_guard import cleaners

## Load data

In [ ]:
files = []
directory = directory + '/'
for file in os.listdir(directory):
    if file.endswith('.ar'):
        files.append(file)
files.sort()
archives = []
archives = [psr.Archive_load(directory + file) for file in files]
print archives[0]
for i in range(1, len(archives)):
    print archives[i]
    archives[0].append(archives[i])

## Display metadata

In [ ]:
nbin = archives[0].get_nbin()
nchan = archives[0].get_nchan()
npol = archives[0].get_npol()
nsubint = archives[0].get_nsubint()
obs_type = archives[0].get_type()
telescope_name = archives[0].get_telescope()
source_name = archives[0].get_source()
ra = archives[0].get_coordinates().ra()
dec = archives[0].get_coordinates().dec()
centre_frequency = archives[0].get_centre_frequency()
bandwidth = archives[0].get_bandwidth()
dm = archives[0].get_dispersion_measure()
rm = archives[0].get_rotation_measure()
is_dedispersed = archives[0].get_dedispersed()
is_faraday_rotated = archives[0].get_faraday_corrected()
is_pol_calib = archives[0].get_poln_calibrated()
data_units = archives[0].get_scale()
data_state = archives[0].get_state()
obs_duration = archives[0].integration_length()
receiver_name = archives[0].get_receiver_name()
receptor_basis = archives[0].get_basis()
backend_name = archives[0].get_backend_name()
low_freq = archives[0].get_centre_frequency() - archives[0].get_bandwidth() / 2.0
high_freq = archives[0].get_centre_frequency() + archives[0].get_bandwidth() / 2.0
subint_duration = np.rint(archives[0].get_Integration(1).get_duration())

In [ ]:
print 'nbin             Number of pulse phase bins                 %s' % nbin
print 'nchan            Number of frequency channels               %s' % nchan
print 'npol             Number of polarizations                    %s' % npol
print 'nsubint          Number of sub-integrations                 %s' % nsubint
print 'type             Observation type                           %s' % obs_type
print 'site             Telescope name                             %s' % telescope_name
print 'name             Source name                                %s' % source_name
print 'coord            Source coordinates                         %s%s' % (ra.getHMS(), dec.getDMS())
print 'freq             Centre frequency (MHz)                     %s' % centre_frequency
print 'bw               Bandwidth (MHz)                            %s' % bandwidth
print 'dm               Dispersion measure (pc/cm^3)               %s' % dm
print 'rm               Rotation measure (rad/m^2)                 %s' % rm
print 'dmc              Dispersion corrected                       %s' % is_dedispersed
print 'rmc              Faraday Rotation corrected                 %s' % is_faraday_rotated
print 'polc             Polarization calibrated                    %s' % is_pol_calib
print 'scale            Data units                                 %s' % data_units
print 'state            Data state                                 %s' % data_state
print 'length           Observation duration (s)                   %s' % obs_duration
print 'rcvr:name        Receiver name                              %s' % receiver_name
print 'rcvr:basis       Basis of receptors                         %s' % receptor_basis
print 'be:name          Name of the backend instrument             %s' % backend_name

## Clean archive, display RFI stats and RFI mask

In [ ]:
clean_rfi = archives[0].clone()
cleaner = cleaners.load_cleaner('surgical')
cleaner.parse_config_string('chan_numpieces=1,subint_numpieces=1,chanthresh=3,subintthresh=3')
cleaner.run(clean_rfi)
cleaner2 = cleaners.load_cleaner('rcvrstd')
cleaner2.parse_config_string('badchans=0:210;286:497;831:886;936:938;1099:1101;1108:1111;1113:1128;1166:1167;1175:1177;1180:1181;1184:1186;1194:1196;1199:1201;1204:1205;1223:1224;1228:1229;1276:1277;1295:1296;1300:1301;1309:1311;1333:1335;1338:1339;1347:1349;1357:1358;1362:1363;1387:2128;2700:2709;2900:3044;3174:3687;3896:4095,badfreqs=None,badsubints=None,trimbw=0,trimfrac=0,trimnum=0,response=None')
cleaner2.run(clean_rfi)
cleaner3 = cleaners.load_cleaner('bandwagon')
cleaner3.parse_config_string('badchantol=0.9,badsubtol=1.0')
cleaner3.run(clean_rfi)
weights = clean_rfi.get_weights()
common_clean_rfi = clean_rfi.clone()
common_clean_rfi.dedisperse()
common_clean_rfi.remove_baseline()
common_clean_rfi.centre_max_bin()
i = j = counter = channel = 0
empty_chan = [i for i in range(nchan)]
for i in range(nsubint):
    channel = 0
    del empty_chan[:]
    for j in range(nchan):
        if weights[i][j] == 0.0:
            counter += 1
            channel += 1
            empty_chan.append(j)
    subint_proc = (float(channel) / float(nchan)) * 100
    print 'Subint %s has %s channels (%.2f%%) set to zero.' % (i, channel, subint_proc)
percentage = (float(counter) / float(weights.size)) * 100
print '%s data points out of %s with weights set to zero.' % (counter, weights.size)
print '%.2f%% data was set to zero.' % (percentage)
fig, ax1 = plt.subplots(1, 1, figsize = [15, 10])
ax1.set_title(source_name)
ax1.set_title('RFI mask', loc='left')
ax1.set_ylabel('Channel number')
ax1.yaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax1.set_xlabel('Subint number')
ax1_secondary = ax1.twinx()
ax1_secondary.set_ylabel('Frequency (MHz)')
ax1_secondary.set_ylim(low_freq, high_freq)
ax1_secondary.yaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 25))
ax1.imshow(weights.T, origin='lower', aspect='auto', cmap=colors.ListedColormap(['red', 'white']), interpolation='none', extent=(0, nsubint - 1, 0, nchan - 1))

## Plot average profiles (Stokes data)

In [ ]:
clean_profile = common_clean_rfi.clone()
clean_profile.tscrunch()
clean_profile.fscrunch()
clean_profile.convert_state('Stokes')
clean_profile_data = clean_profile.get_data()
clean_profile_data = clean_profile_data.squeeze()
clean_min_max_vals = np.array([clean_profile_data[0, :].max(), clean_profile_data[1, :].max(), clean_profile_data[2, :].max(), clean_profile_data[3, :].max(), clean_profile_data[0, :].min(), clean_profile_data[1, :].min(), clean_profile_data[2, :].min(), clean_profile_data[3, :].min()])
dirty_profile = archives[0].clone()
dirty_profile.dedisperse()
dirty_profile.remove_baseline()
dirty_profile.centre_max_bin()
dirty_profile.tscrunch()
dirty_profile.fscrunch()
dirty_profile.convert_state('Stokes')
dirty_profile_data = dirty_profile.get_data()
dirty_profile_data = dirty_profile_data.squeeze()
dirty_min_max_vals = np.array([dirty_profile_data[0, :].min(), dirty_profile_data[1, :].min(), dirty_profile_data[2, :].min(), dirty_profile_data[3, :].min(), dirty_profile_data[0, :].max(), dirty_profile_data[1, :].max(), dirty_profile_data[2, :].max(), dirty_profile_data[3, :].max()])
fig, ((ax1, ax2)) = plt.subplots(2, 1, sharex='col', figsize = [15, 10], tight_layout='true')
ax1.set_title(source_name)
ax1.text(20, dirty_profile_data.max(), 'Unclean data', fontsize='large')
ax1.set_ylabel('Flux (a.u.)')
ax1.set_ylim((dirty_min_max_vals.min() * 1.2), dirty_min_max_vals.max() + 0.1 * dirty_min_max_vals.max())
ax1.plot(dirty_profile_data[0, :], 'k', label='I')
ax1.plot(dirty_profile_data[1, :], 'r', label='Q')
ax1.plot(dirty_profile_data[2, :], 'g', label='U')
ax1.plot(dirty_profile_data[3, :], 'b', label='V')
ax1.legend()
ax2.text(20, clean_profile_data.max(), 'Clean data', fontsize='large')
ax2.set_xlabel('Pulse phase (bins)')
ax2.set_xlim(0, nbin - 1)
ax2.set_ylabel('Flux (a.u.)')
ax2.set_ylim((clean_min_max_vals.min() * 1.2), clean_min_max_vals.max() + 0.1 * clean_min_max_vals.max())
ax2.xaxis.set_ticks(np.arange(0, nbin - 1, 50))
ax2_secondary = ax2.twiny()
ax2_secondary.set_frame_on(True)
ax2_secondary.patch.set_visible(False)
ax2_secondary.xaxis.set_ticks_position('bottom')
ax2_secondary.set_xlabel('Pulse phase (degrees)')
ax2_secondary.xaxis.set_label_position('bottom')
ax2_secondary.spines['bottom'].set_position(('outward', 50))
ax2_secondary.set_xlim(0, 360)
_ = ax2_secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax2.plot(clean_profile_data[0, :], 'k', label='I')
ax2.plot(clean_profile_data[1, :], 'r', label='Q')
ax2.plot(clean_profile_data[2, :], 'g', label='U')
ax2.plot(clean_profile_data[3, :], 'b', label='V')
ax2.legend()

## Plot subintegration stacks (Coherency data)

In [ ]:
dirty_stack = archives[0].clone()
dirty_stack.dedisperse()
dirty_stack.remove_baseline()
dirty_stack.centre_max_bin()
dirty_stack.fscrunch()
dirty_stack.pscrunch()
dirty_stack_data = dirty_stack.get_data().squeeze()
clean_stack = common_clean_rfi.clone()
clean_stack.fscrunch()
clean_stack.pscrunch()
clean_stack_data = clean_stack.get_data().squeeze()
fig, ((ax1, ax2)) = plt.subplots(1, 2, sharey='row', figsize=[15, 10], tight_layout='true')
ax1.set_title(source_name)
ax1.set_title('Unclean data', loc='left')
ax1.set_ylabel('Subint number')
ax1.set_xlabel('Pulse phase (bin)')
ax1.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax1_secondary = ax1.twiny()
ax1_secondary.set_frame_on(True)
ax1_secondary.patch.set_visible(False)
ax1_secondary.xaxis.set_ticks_position('bottom')
ax1_secondary.set_xlabel('Pulse phase (degrees)')
ax1_secondary.xaxis.set_label_position('bottom')
ax1_secondary.spines['bottom'].set_position(('outward', 50))
ax1_secondary.set_xlim(0, 360)
_ = ax1_secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax1.imshow(dirty_stack_data, cmap=cm.afmhot, aspect='auto', vmax=dirty_stack_data.max() * 0.75, interpolation='none', extent=(0, nbin - 1, 0, nsubint - 1))
ax2.set_title(source_name)
ax2.set_title('Clean data', loc='left')
ax2.set_xlabel('Pulse phase (bin)')
ax2.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax2_secondary = ax2.twiny()
ax2_secondary.set_frame_on(True)
ax2_secondary.patch.set_visible(False)
ax2_secondary.xaxis.set_ticks_position('bottom')
ax2_secondary.set_xlabel('Pulse phase (degrees)')
ax2_secondary.xaxis.set_label_position('bottom')
ax2_secondary.spines['bottom'].set_position(('outward', 50))
ax2_secondary.set_xlim(0, 360)
_ = ax2_secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax2_tertiary = ax2.twinx()
ax2_tertiary.yaxis.set_label_position('right')
ax2_tertiary.set_ylim(0, obs_duration)
ax2_tertiary.set_ylabel('Time (seconds)')
ax2.imshow(clean_stack_data, cmap=cm.afmhot, aspect='auto', vmax=clean_stack_data.max() * 0.75, interpolation='none', extent=(0, nbin - 1, 0, nsubint - 1))

## Plot phase vs. frequency image of flux (clean Coherency data)

In [ ]:
coherency_flux_phase = common_clean_rfi.clone()
coherency_flux_phase.tscrunch()
coherency_flux_phase_data = coherency_flux_phase.get_data().squeeze()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.015)
fig.text(0.05, 1.0, 'Clean data', fontsize='large')
ax1.set_title('XX')
ax1.set_ylabel('Frequency (MHz)')
ax1.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax1.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax1.imshow(coherency_flux_phase_data[0, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))
ax2.set_title('YY')
ax2.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax2.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax2_secondary = ax2.twinx()
ax2_secondary.set_ylabel('Channel number')
ax2_secondary.set_ylim(0, nchan - 1)
ax2_secondary.yaxis.set_ticks(np.arange(0, nchan - 1 , 200))
ax2.imshow(coherency_flux_phase_data[1, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))
ax3.set_title('Re XY')
ax3.set_xlabel('Pulse phase (bin)')
ax3.set_ylabel('Frequency (MHz)')
ax3.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax3.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax3_secondary = ax3.twiny()
ax3_secondary.set_frame_on(True)
ax3_secondary.patch.set_visible(False)
ax3_secondary.xaxis.set_ticks_position('bottom')
ax3_secondary.set_xlabel('Pulse phase (degrees)')
ax3_secondary.xaxis.set_label_position('bottom')
ax3_secondary.spines['bottom'].set_position(('outward', 50))
ax3_secondary.set_xlim(0, 360)
_ = ax3_secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax3.imshow(coherency_flux_phase_data[2, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))
ax4.set_title('Im XY')
ax4.set_xlabel('Pulse phase (bin)')
ax4.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax4.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax4_secondary = ax4.twinx()
ax4_secondary.set_ylabel('Channel number')
ax4_secondary.set_ylim(0, nchan - 1)
ax4_secondary.yaxis.set_ticks(np.arange(0, nchan - 1 , 200))
ax4_tertiary = ax4.twiny()
ax4_tertiary.set_frame_on(True)
ax4_tertiary.patch.set_visible(False)
ax4_tertiary.xaxis.set_ticks_position('bottom')
ax4_tertiary.set_xlabel('Pulse phase (degrees)')
ax4_tertiary.xaxis.set_label_position('bottom')
ax4_tertiary.spines['bottom'].set_position(('outward', 50))
ax4_tertiary.set_xlim(0, 360)
_ = ax4_tertiary.xaxis.set_ticks(np.arange(0, 360, 20))
ax4.imshow(coherency_flux_phase_data[3, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))

## Plot phase vs. frequency image of flux (clean Stokes data)

In [ ]:
stokes_flux_phase = common_clean_rfi.clone()
stokes_flux_phase.tscrunch()
stokes_flux_phase.convert_state('Stokes')
stokes_flux_phase_data = stokes_flux_phase.get_data().squeeze()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize='large')
ax1.set_title('I')
ax1.set_ylabel('Frequency (MHz)')
ax1.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax1.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax1.imshow(stokes_flux_phase_data[0, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))
ax2.set_title('Q')
ax2.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax2.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax2_secondary = ax2.twinx()
ax2_secondary.set_ylabel('Channel number')
ax2_secondary.set_ylim(0, nchan - 1)
ax2_secondary.yaxis.set_ticks(np.arange(0, nchan - 1 , 200))
ax2.imshow(stokes_flux_phase_data[1, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))
ax3.set_title('U')
ax3.set_xlabel('Pulse phase (bin)')
ax3.set_ylabel('Frequency (MHz)')
ax3.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax3.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax3_secondary = ax3.twiny()
ax3_secondary.set_frame_on(True)
ax3_secondary.patch.set_visible(False)
ax3_secondary.xaxis.set_ticks_position('bottom')
ax3_secondary.set_xlabel('Pulse phase (degrees)')
ax3_secondary.xaxis.set_label_position('bottom')
ax3_secondary.spines['bottom'].set_position(('outward', 50))
ax3_secondary.set_xlim(0, 360)
_ = ax3_secondary.xaxis.set_ticks(np.arange(0, 360, 20))
ax3.imshow(stokes_flux_phase_data[2, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))
ax4.set_title('V')
ax4.set_xlabel('Pulse phase (bin)')
ax4.xaxis.set_ticks(np.arange(0, nbin - 1, 100))
ax4.yaxis.set_ticks(np.arange(0, nchan - 1, 50))
ax4_secondary = ax4.twinx()
ax4_secondary.set_ylabel('Channel number')
ax4_secondary.set_ylim(0, nchan - 1)
ax4_secondary.yaxis.set_ticks(np.arange(0, nchan - 1 , 200))
ax4_tertiary = ax4.twiny()
ax4_tertiary.set_frame_on(True)
ax4_tertiary.patch.set_visible(False)
ax4_tertiary.xaxis.set_ticks_position('bottom')
ax4_tertiary.set_xlabel('Pulse phase (degrees)')
ax4_tertiary.xaxis.set_label_position('bottom')
ax4_tertiary.spines['bottom'].set_position(('outward', 50))
ax4_tertiary.set_xlim(0, 360)
_ = ax4_tertiary.xaxis.set_ticks(np.arange(0, 360, 20))
ax4.imshow(stokes_flux_phase_data[3, :, :], cmap=cm.afmhot, aspect='auto', interpolation='none', extent=(0, nbin - 1.0, low_freq, high_freq))

## Plot bandpass (unclean Stokes data)

In [ ]:
dirty_bandpass = archives[0].clone()
dirty_bandpass.tscrunch()
(dirty_bandpass_mean, dirty_bandpass_variance) = dirty_bandpass.get_Integration(0).baseline_stats()
dirty_bandpass_max_min_vals = np.array([dirty_bandpass_mean[0, :].min(), dirty_bandpass_mean[0, :].max(), dirty_bandpass_mean[1, :].min(), dirty_bandpass_mean[1, :].max()])
dirty_bandpass_stokes = archives[0].clone()
dirty_bandpass_stokes.convert_state('Stokes')
dirty_bandpass_stokes.tscrunch()
(dirty_bandpass_stokes_mean, dirty_bandpass_stokes_variance) = dirty_bandpass_stokes.get_Integration(0).baseline_stats()
dirty_bandpass_stokes_min_max_vals = np.array([dirty_bandpass_stokes_mean[0, :].min(), dirty_bandpass_stokes_mean[0, :].max()])
fig, ((ax1, ax2, ax3 )) = plt.subplots(3, 1, sharex='col', figsize=[15, 10], tight_layout='true')
ax1.set_title(source_name)
ax1.set_title('Unclean data', loc='left')
ax1.text(np.int(nchan * 0.0125), dirty_bandpass_max_min_vals.max() - (0.25 * dirty_bandpass_max_min_vals.max()), 'XX', fontsize='large')
ax1.set_xlim(0, nchan - 1)
ax1.set_ylabel('Flux (a.u.)')
ax1.xaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax1.set_yscale('log')
ax1.plot(dirty_bandpass_mean[0, :])
ax2.text(np.int(nchan * 0.0125), dirty_bandpass_max_min_vals.max() - (0.25 * dirty_bandpass_max_min_vals.max()), 'YY', fontsize='large')
ax2.set_xlim(0, nchan - 1)
ax2.set_ylabel('Flux (a.u.)')
ax2.xaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax2.set_yscale('log')
ax2.plot(dirty_bandpass_mean[1, :])
ax3.text(np.int(nchan*0.0125), dirty_bandpass_max_min_vals.max() - (0.25 * dirty_bandpass_max_min_vals.max()), 'Stokes I', fontsize='large')
ax3.set_xlabel('Channel number')
ax3.set_ylabel('Flux (a.u.)')
ax3.xaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax3.set_yscale('log')
ax3_secondary = ax3.twiny()
ax3_secondary.set_frame_on(True)
ax3_secondary.patch.set_visible(False)
ax3_secondary.xaxis.set_ticks_position('bottom')
ax3_secondary.set_xlabel('Frequency (MHz)')
ax3_secondary.xaxis.set_label_position('bottom')
ax3_secondary.spines['bottom'].set_position(('outward', 50))
ax3_secondary.set_xlim(low_freq, high_freq)
_ = ax3_secondary.xaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 50))
ax3.plot(dirty_bandpass_stokes_mean[0, :])

## Plot bandpass (clean Stokes data)

In [ ]:
clean_bandpass = clean_rfi.clone()
clean_bandpass.tscrunch()
(clean_bandpass_mean, clean_bandpass_variance) = clean_bandpass.get_Integration(0).baseline_stats()
clean_bandpass_max_min_vals = np.array([clean_bandpass_mean[0, :].min(), clean_bandpass_mean[0, :].max(), clean_bandpass_mean[1, :].min(), clean_bandpass_mean[1, :].max()])
clean_bandpass_mean[clean_bandpass_mean == 0.0] = np.nan
clean_bandpass_stokes = clean_bandpass.clone()
clean_bandpass_stokes.convert_state('Stokes')
clean_bandpass_stokes.tscrunch()
(clean_bandpass_stokes_mean, clean_bandpass_stokes_variance) = clean_bandpass_stokes.get_Integration(0).baseline_stats()
clean_bandpass_stokes_min_max_vals = np.array([clean_bandpass_stokes_mean[0, :].min(), clean_bandpass_stokes_mean[0, :].max()])
clean_bandpass_stokes_mean[clean_bandpass_stokes_mean == 0.0] = np.nan
fig, ((ax1, ax2, ax3 )) = plt.subplots(3, 1, sharex='col', figsize=[15, 10], tight_layout='true')
ax1.set_title(source_name)
ax1.set_title('Clean data', loc='left')
ax1.text(np.int(nchan * 0.0125), clean_bandpass_max_min_vals.max(), 'XX', fontsize='large')
ax1.set_xlim(0, nchan - 1)
ax1.set_ylabel('Flux (a.u.)')
ax1.set_ylim(clean_bandpass_max_min_vals.min() - (0.1 * np.abs(clean_bandpass_max_min_vals.min())), clean_bandpass_max_min_vals.max() + (0.1 * np.abs(clean_bandpass_max_min_vals.max())))
ax1.xaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax1.plot(clean_bandpass_mean[0, :])
ax2.text(np.int(nchan * 0.0125), clean_bandpass_max_min_vals.max(), 'YY', fontsize='large')
ax2.set_xlim(0, nchan - 1)
ax2.set_ylabel('Flux (a.u.)')
ax2.set_ylim(clean_bandpass_max_min_vals.min() - (0.1 * np.abs(clean_bandpass_max_min_vals.min())), clean_bandpass_max_min_vals.max() + (0.1 * np.abs(clean_bandpass_max_min_vals.max())))
ax2.xaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax2.plot(clean_bandpass_mean[1, :])
ax3.text(np.int(nchan * 0.0125), clean_bandpass_stokes_min_max_vals.max(), 'Stokes I', fontsize='large')
ax3.set_xlabel('Channel number')
ax3.set_ylabel('Flux (a.u.)')
ax3.set_ylim(clean_bandpass_stokes_min_max_vals.min() - (0.1 * np.abs(clean_bandpass_stokes_min_max_vals.min())), clean_bandpass_stokes_min_max_vals.max() + (0.1 * np.abs(clean_bandpass_stokes_min_max_vals.max())))
ax3.xaxis.set_ticks(np.arange(0, nchan - 1, 200))
ax3_secondary = ax3.twiny()
ax3_secondary.set_frame_on(True)
ax3_secondary.patch.set_visible(False)
ax3_secondary.xaxis.set_ticks_position('bottom')
ax3_secondary.set_xlabel('Frequency (MHz)')
ax3_secondary.xaxis.set_label_position('bottom')
ax3_secondary.spines['bottom'].set_position(('outward', 50))
ax3_secondary.set_xlim(low_freq, high_freq)
_ = ax3_secondary.xaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 50))
ax3.plot(clean_bandpass_stokes_mean[0, :])

## Plot dynamic baseline spectrum (unclean Coherency data)

In [ ]:
dynamic_spectrum = archives[0].clone()
mean = np.zeros((nsubint, npol, nchan))
variance = np.zeros((nsubint, npol, nchan))
for subint in range(nsubint):
    m, v = dynamic_spectrum.get_Integration(subint).baseline_stats()
    mean[subint] = m[:, :]
mean_xx_min = mean[:, 0, :].min()
mean_xx_max = mean[:, 0, :].max()
mean_yy_min = mean[:, 1, :].min()
mean_yy_max = mean[:, 1, :].max()
mean_re_xy_min = mean[:, 2, :].min()
mean_re_xy_max = mean[:, 2, :].max()
mean_im_xy_min = mean[:, 3, :].min()
mean_im_xy_max = mean[:, 3, :].max()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Unclean data', fontsize='large')
ax1.set_title('XX')
ax1.set_ylabel('Channel number')
ax1.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax1.imshow(mean[:, 0, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.SymLogNorm(10, linscale=1.0, vmin=mean_xx_min, vmax=mean_xx_max), interpolation='none')
ax2.set_title('YY')
ax2.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax2_secondary = ax2.twinx()
ax2_secondary.set_ylabel('Frequency (MHz)')
ax2_secondary.set_ylim(low_freq, high_freq)
ax2_secondary.yaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 50))
ax2.imshow(mean[:, 1, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.SymLogNorm(10, linscale=1.0, vmin=mean_yy_min, vmax=mean_yy_max), interpolation='none')
ax3.set_title('Re XY')
ax3.set_xlabel('Subint number')
ax3.set_ylabel('Channel number')
ax3.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax3_secondary = ax3.twiny()
ax3_secondary.set_frame_on(True)
ax3_secondary.patch.set_visible(False)
ax3_secondary.xaxis.set_ticks_position('bottom')
ax3_secondary.set_xlabel('Time (seconds)')
ax3_secondary.xaxis.set_label_position('bottom')
ax3_secondary.spines['bottom'].set_position(('outward', 50))
ax3_secondary.set_xlim(0, obs_duration)
ax3.imshow(mean[:, 2, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.SymLogNorm(10, linscale=1.0, vmin=mean_re_xy_min, vmax=mean_re_xy_max), interpolation='none')
ax4.set_title('Im XY')
ax4.set_xlabel('Subint number')
ax4.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
aax4_secondary = ax4.twinx()
aax4_secondary.set_ylabel('Frequency (MHz)')
aax4_secondary.set_ylim(low_freq, high_freq)
aax4_secondary.yaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 50))
ax4_tertiary = ax4.twiny()
ax4_tertiary.set_frame_on(True)
ax4_tertiary.patch.set_visible(False)
ax4_tertiary.xaxis.set_ticks_position('bottom')
ax4_tertiary.set_xlabel('Time (seconds)')
ax4_tertiary.xaxis.set_label_position('bottom')
ax4_tertiary.spines['bottom'].set_position(('outward', 50))
ax4_tertiary.set_xlim(0, obs_duration)
ax4.imshow(mean[:, 3, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.SymLogNorm(10, linscale=1.0, vmin=mean_im_xy_min, vmax=mean_im_xy_max), interpolation='none')

## Plot dynamic baseline spectrum (clean Stokes data)

In [ ]:
dynamic_spectrum = clean_rfi.clone()
dynamic_spectrum.convert_state('Stokes')
mean = np.zeros((nsubint, npol, nchan))
variance = np.zeros((nsubint, npol, nchan))
for subint in range(nsubint):
    m, v = dynamic_spectrum.get_Integration(subint).baseline_stats()
    variance[subint] = v[:, :]
variance_xx_min = variance[:, 0, :].min()
variance_xx_max = variance[:, 0, :].max()
variance_yy_min = variance[:, 1, :].min()
variance_yy_max = variance[:, 1, :].max()
variance_re_xy_min = variance[:, 2, :].min()
variance_re_xy_max = variance[:, 2, :].max()
variance_im_xy_min = variance[:, 3, :].min()
variance_im_xy_max = variance[:, 3, :].max()
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize='large')
ax1.set_title('I')
ax1.set_ylabel('Channel number')
ax1.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax1.imshow(variance[:, 0, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.Normalize(vmin=variance_xx_min, vmax=variance_xx_max), interpolation='none')
ax2.set_title('Q')
ax2.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax2_secondary = ax2.twinx()
ax2_secondary.set_ylabel('Frequency (MHz)')
ax2_secondary.set_ylim(low_freq, high_freq)
ax2_secondary.yaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 50))
ax2.imshow(variance[:, 1, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.Normalize(vmin=variance_yy_min, vmax=variance_yy_max), interpolation='none')
ax3.set_title('U')
ax3.set_xlabel('Subint number')
ax3.set_ylabel('Channel number')
ax3.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax3_secondary = ax3.twiny()
ax3_secondary.set_frame_on(True)
ax3_secondary.patch.set_visible(False)
ax3_secondary.xaxis.set_ticks_position('bottom')
ax3_secondary.set_xlabel('Time (seconds)')
ax3_secondary.xaxis.set_label_position('bottom')
ax3_secondary.spines['bottom'].set_position(('outward', 50))
ax3_secondary.set_xlim(0, obs_duration)
ax3.imshow(variance[:, 2, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.Normalize(vmin=variance_re_xy_min, vmax=variance_re_xy_max), interpolation='none')
ax4.set_title('V')
ax4.set_xlabel('Subint number')
ax4.yaxis.set_ticks(np.arange(0, nchan - 1, 400))
ax4_secondary = ax4.twinx()
ax4_secondary.set_ylabel('Frequency (MHz)')
ax4_secondary.set_ylim(low_freq, high_freq)
ax4_secondary.yaxis.set_ticks(np.arange(np.rint(low_freq), np.rint(high_freq), 50))
ax4_tertiary = ax4.twiny()
ax4_tertiary.set_frame_on(True)
ax4_tertiary.patch.set_visible(False)
ax4_tertiary.xaxis.set_ticks_position('bottom')
ax4_tertiary.set_xlabel('Time (seconds)')
ax4_tertiary.xaxis.set_label_position('bottom')
ax4_tertiary.spines['bottom'].set_position(('outward', 50))
ax4_tertiary.set_xlim(0, obs_duration)
ax4.imshow(variance[:, 3, :].T, origin='lower', aspect='auto', cmap=cm.afmhot, norm=colors.Normalize(vmin=variance_im_xy_min, vmax=variance_im_xy_max), interpolation='none')

## Plot signal-to-noise ratio vs. subintegration (clean Coherency data)

In [ ]:
archive_snr = common_clean_rfi.clone()
new_integration = archive_snr.get_Integration(0).total()
snr_data = np.zeros(nsubint)
for i_subintegration in range (1, nsubint):
    next_integration = archive_snr.get_Integration(i_subintegration).total()
    new_integration.combine(next_integration)
    profile = new_integration.get_Profile(0, 0)
    snr_data[i_subintegration] = profile.snr()
fig, ax1 = plt.subplots(1, 1, sharex='col', sharey='row', figsize=[15, 5], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize='large')
ax1.set_xlabel('Subint number')
ax1.set_ylabel('Signal-to-noise ratio')
ax1.set_xlim(0, nsubint)
ax1_secondary = ax1.twiny()
ax1_secondary.set_frame_on(True)
ax1_secondary.patch.set_visible(False)
ax1_secondary.xaxis.set_ticks_position('bottom')
ax1_secondary.set_xlabel('Time (seconds)')
ax1_secondary.xaxis.set_label_position('bottom')
ax1_secondary.spines['bottom'].set_position(('outward', 50))
ax1_secondary.set_xlim(0, obs_duration)
ax1.plot(snr_data)

In [ ]:
print 'Report Ended:'
!date